In [1]:
from scipy.stats import moment
from numpy import mean, var
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import random

import matplotlib.colors as colors
from sklearn.utils import resample
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
from sklearn.ensemble import GradientBoostingClassifier

In [2]:
# reading the data
data = pd.read_csv('bounded_c.csv', index_col=None)

# predictors
X = data.drop('dist', axis=1)

# Scaling data
scaler = StandardScaler()
scaler.fit(X)
X_scaled = scaler.transform(X)

# response variable
y = data['dist']

# train-test split
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size= 0.2, random_state=10)

In [6]:
param_grid = [
    {'C':np.logspace(0.1, 50, 20),
     'gamma':np.logspace(0.001, 10, 20), 
     'kernel':['rbf']},
]

optimal_params = GridSearchCV(SVC(), param_grid,cv=5, verbose=0)

#optimal_params.fit(X_train, y_train)
print(optimal_params.best_params_)

c = optimal_params.best_params_['C']
g = optimal_params.best_params_['gamma']

clf_svm = SVC(random_state=10, C=c, gamma=g)
clf_svm.fit(X_train, y_train)

y_pred = clf_svm.predict(X_test)

c_matrix = confusion_matrix(y_test, y_pred)

disp = ConfusionMatrixDisplay(c_matrix, display_labels=clf_svm.classes_)
disp.plot(cmap=plt.cm.Blues, colorbar=False, xticks_rotation='vertical')
plt.show()

AttributeError: 'GridSearchCV' object has no attribute 'best_params_'

In [38]:
labels = clf_svm.classes_
print(classification_report(y_test, y_pred, target_names=labels))

              precision    recall  f1-score   support

     arcsine       1.00      1.00      1.00         6
        beta       1.00      1.00      1.00         4
    powerlaw       1.00      1.00      1.00         7
   trapezoid       1.00      1.00      1.00         6
  triangular       1.00      1.00      1.00         4
     uniform       1.00      1.00      1.00         3

    accuracy                           1.00        30
   macro avg       1.00      1.00      1.00        30
weighted avg       1.00      1.00      1.00        30



In [40]:
scores = cross_val_score(clf_svm, X, y, cv=5)
scores

array([1., 1., 1., 1., 1.])